# Aquiring Celestial Data

## Spice


The Navigation and Ancillary Information Facility (NAIF), acting under the directions of NASA's Planetary Science Division, has built an information system named "SPICE" to assist NASA scientists in planning and interpreting scientific observations from space-borne instruments, and to assist NASA engineers involved in modeling, planning and executing activities needed to conduct planetary exploration missions. [[source](https://naif.jpl.nasa.gov/naif/index.html)]

`tudat` provides a nonextensive interface to the cspice library. The interface can be accessed with the following import using `tudatpy`:

In [1]:
from tudatpy.kernel.interface import spice_interface

ModuleNotFoundError: No module named 'tudatpy'

`tudat` has a list of curated default kernels which can be used for most general applications of space mission analysis and design. The default kernel paths can be retrieved and inspected through the following function:

In [17]:
# inspect tudat standard spice kernels
spice_interface.get_standard_kernels([])

['/home/ggarrett/.tudat/resource/spice_kernels/pck00010.tpc',
 '/home/ggarrett/.tudat/resource/spice_kernels/gm_de431.tpc',
 '/home/ggarrett/.tudat/resource/spice_kernels/tudat_merged_spk_kernel.bsp',
 '/home/ggarrett/.tudat/resource/spice_kernels/naif0012.tls']

The default kernels can be loaded directly into the `spice` kernel pool as follows:

In [43]:
# load tudat standard spice kernels
spice_interface.load_standard_kernels()
# count of loaded kernels
spice_interface.get_total_count_of_kernels_loaded()

In case it is required, the loaded kernels into the `spice` kernel are deleted with the function:

In [47]:
# delete the loaded kernels
# spice_interface.clear_kernels()

0

Another way to carry out the previous steps:

In [8]:
import numpy as np
import spiceypy as spice
import os

# required internal imports
from tudatpy.kernel import io
from tudatpy.kernel.astro import two_body_dynamics

# current implementation
tudat_spice_kernels = [
    "pck00010.tpc",
    "gm_de431.tpc",
    "tudat_merged_spk_kernel.bsp",
    "naif0012.tls"]

spice_kernel_paths = map(
    lambda x: os.path.join(io.get_spice_kernel_path(), x),
    tudat_spice_kernels)

# future implementation
# spice_kernel_paths = spice_interface.get_standard_kernels()

# load standard spice kernels
# spice.furnsh(spice_kernel_paths)

0

In [10]:
# spice.kclear()

# spice.ktotal("ALL")

# spice.bodvrd("Earth", "GM", 1)

spice.failed()

False

In [13]:
# spice.swpool()

from ctypes.util import find_library

find_library("cspice")

'/home/ggarrett/miniconda3/envs/tudat-env/lib/libcspice.so.66'

# Time conversions

## Concepts

#### Terrestial Time (TT)

Terrestrial Time (TT) is a modern astronomical time standard defined by the International Astronomical Union, primarily for time-measurements of astronomical observations made from the surface of Earth.[1] For example, the Astronomical Almanac uses TT for its tables of positions (ephemerides) of the Sun, Moon and planets as seen from Earth. In this role, TT continues Terrestrial Dynamical Time (TDT or TD),[2] which in turn succeeded ephemeris time (ET). TT shares the original purpose for which ET was designed, to be free of the irregularities in the rotation of Earth.

#### Barycentric Dynamical Time (TDB)

Barycentric Dynamical Time (TDB, from the French Temps Dynamique Barycentrique) is a relativistic coordinate time scale, intended for astronomical use as a time standard to take account of time dilation [1] 

## Useful Examples

Convert from Julian date to Ephemeris time

In [52]:
spice_interface.convert_julian_date_to_ephemeris_time(0)

-211813488000.0

Convert from Ephemeris time to Julian date

In [54]:
spice_interface.convert_ephemeris_time_to_julian_date(0)

2451545.0

Convert a date written in a string to Ephemeris time

In [ ]:
str_test = "1988 June 13, 12:29:48 A.M."
spice_interface.convert_date_string_to_ephemeris_time(str_test)

Another way to carry out the previous steps:

In [14]:
import spiceypy as spice
from tudatpy.kernel.interface import spice_interface


kernels = spice_interface.get_standard_kernels([])
spice_interface.load_standard_kernels()

spice.furnsh(kernels)
# spice.str2et(str_test)

str_test = "1988 June 13, 12:29:48 A.M."

spice_interface.convert_date_string_to_ephemeris_time(str_test)



-364562955.81541026

# Body parameters

Get the gravitational parameter of a body

In [14]:
spice_interface.get_body_gravitational_parameter("Venus")

398600435436095.94

Get the gravitational parameter of a body using spice bodvrd routine

In [16]:
spice_interface.get_body_properties("Saturn", "GM", 1)

[398600.4354360959]

Get the principal elliptical axes of a body using spice bodvrd routine

In [21]:
spice_interface.get_body_properties("Mars", "RADII", 3)

[6378.1366, 6378.1366, 6356.7519]

Get the aritmetic average radius of a bodies principal elliptical axes

In [22]:
spice_interface.get_average_radius("Jupiter")

6371008.366666666

# State Conversions

In [18]:
from tudatpy import elements
import numpy as np

## Spherical to Cartesian

In [22]:
help(elements.spherical2cartesian)

Help on function spherical2cartesian in module tudatpy.elements:

spherical2cartesian(r, lat, lon, speed, fpa, heading) -> numpy.ndarray
    Function to convert spherical state to cartesian.
    
    Parameters
    ----------
    r : float
        Position vector magnitude (m).
    lat : float
        Latitude (m).
    lon : float
        Longitude (m).
    speed : float
        Magnitude of velocity (m).
    fpa : float
        Flight path angle (rad).
    heading : float
        Heading angle (rad).
    
    Returns
    -------
    cartesian_state : ndarray
        Cartesian state represented as [Rx, Ry, Rz, Vx, Vz, Vy] with distance in (m) and speed in (m/s).



In [ ]:
rE = spice_interface.get_average_radius("Earth")
elements.spherical2cartesian(rE+500e3,np.deg2rad(-110),np.deg2rad(-20),7.2e3,
                             np.deg2rad(30),np.deg2rad(150))

## Keplerian to Cartesian

In [23]:
help(elements.keplerian2cartesian)

Help on function keplerian2cartesian in module tudatpy.elements:

keplerian2cartesian(mu: float = None, sma: float = None, ecc: float = None, inc: float = None, raan: float = None, argp: float = None, theta: float = None, **kwargs) -> numpy.ndarray
    Function to convert Keplerian state to cartesian.
    
    Parameters
    ----------
    mu : float
        Standard gravitational parameter (m^3 / s^2).
        (alias = "gravitational_parameter")
    sma : float
        Semi-major axis (m).
        (alias = "semi_major_axis")
    ecc : float
        Eccentricity (-).
        (alias = "eccentricity")
    inc : float
        Inclination (rad).
        (alias = "inclination")
    raan : float
        Right Ascension of the Ascending Node (rad).
        (alias = "right_ascension_of_the_ascending_node")
    argp : float
        Argument of Perigee (rad).
        (alias = "argument_of_periapsis")
    theta : float
        True anomaly (rad).
        (alias = "true_anomaly")
    
    Returns


In [ ]:
mu = spice_interface.get_body_gravitational_parameter("Earth")
elements.keplerian2cartesian(mu,9505e3,0.079,np.deg2rad(7),np.deg2rad(4),
                             np.deg2rad(272.8),np.deg2rad(128.4))

## Body states

In [ ]:
spice_interface.get_body_cartesian_position_at_epoch("Earth","Sun","ECLIPJ2000","NONE",0.81541026)

In [ ]:
spice_interface.get_body_cartesian_state_at_epoch("Earth","Sun","ECLIPJ2000","NONE",0.81541026)

In [ ]:
spice_interface.get_cartesian_state_from_tle_at_epoch()